<a href="https://colab.research.google.com/github/GreihMurray/NLP-3/blob/Super_Murray/supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Reshape
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
import joblib
from sklearn.metrics import accuracy_score
import pickle
import nltk
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [22]:
def read_file_to_sents():
    all_data = []
    with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/train.tsv", encoding="utf-8") as file:
        f = csv.reader(file, delimiter="\t")
        for line in tqdm(f, desc="Reading data..."):
            word = line[0]
            graphemes = line[1].split('-')

            cur_word = []

            for i in range(0, len(graphemes)):
                if len(graphemes[i]) == 1:
                    cur_word.append((word[i], 'B'))
                else:
                    cur_word.append((word[i], 'B'))
                    for j in range(i+1, (i + len(graphemes[i]))):
                        cur_word.append((word[j], 'I'))
                        i += j

            all_data.append(cur_word)

    return all_data

In [4]:
def supervised():
    data = read_file_to_sents()

In [21]:
supervised()

Reading data...: 12812it [00:00, 178316.38it/s]

ounfò ['oun', 'f', 'ò'] [('o', 'B'), ('u', 'I'), ('n', 'I'), ('u', 'B'), ('n', 'B')]
